## Importing to MNE 

In [ ]:
import PyQt5
%matplotlib qt
%matplotlib inline
import mne
import numpy as np
import pandas as pd 
from mne import io, read_proj, read_selection
from mne.time_frequency import tfr_morlet, psd_multitaper, psd_welch
from mne.datasets import somato
import matplotlib.pyplot as plt


### Loading Data

In [ ]:
### Main time segments of data 

# awake: 400,000 to 900,000 ms
# light: 950,000 to 1,100,000 ms
# general: 1,100,000 to 1,400,000 ms
# deep: 1,500,000 to 2,000,000 ms

data = pd.read_csv('light.csv', header=None, usecols=[*range(0,150000)]) 
ch1 = ['FEF','vlPFC','PPC','STG','RAntThal all Bipolars', 
            'RPostThal all Bipolars','LAntThal all Bipolars','LPostThal all Bipolars']
n_channels = 8
sfreq = 1000 

info = mne.create_info(ch_names = ch1, sfreq = sfreq)
raw = mne.io.RawArray(data, info)

#row 2 is with EEG parameter
info2 = mne.create_info(ch_names = ch1, sfreq = sfreq, ch_types='eeg')
raw2 = mne.io.RawArray(data, info2)


raw2.info

In [ ]:
raw2.plot()

## Pre Processing


#### Fast Fourier Transform


In [ ]:

fmin, fmax = 2, 60  # look at frequencies between 2 and 300Hz
n_fft = 2048  # the FFT size (n_fft). Ideally a power of 2

picks1 = mne.pick_types(raw2.info, eeg=True, eog=False,
                       stim=False, exclude='bads')

fft= raw2.plot_psd(area_mode='range', tmax=10.0, picks=picks1, average=False)

#### Remove power line noise with notch filtering 

In [ ]:
raw2.notch_filter(np.arange(60, 241, 60), picks=picks1, filter_length='auto',
                 phase='zero')

notch = raw2.plot_psd(area_mode='range', tmax=10.0, picks=picks1, average=False)

#### Remove power line noise with low pass filtering 

In [ ]:
# low pass filtering below 50 Hz
raw2.filter(None, 50., fir_design='firwin')  # re-do in the morning 

low_pass = raw2.plot_psd(area_mode='range', tmax=10.0,picks=picks1, average=False)

#### Apply Band Pass Filter - applies both high pass and low pass filters

In [ ]:
raw2.filter(1, 50., fir_design='firwin')

bandpass= raw2.plot_psd(area_mode='range', tmax=10.0,picks=picks1, average=False)

#### Using multitaper spectrum estimation with 7 DPSS windows


In [ ]:
events = mne.find_events(raw2, stim_channel='vlPFC', shortest_event=1 )

epochs = mne.Epochs(raw2,events, picks=picks1,
                    preload=True)

epochs.plot_psd(fmin=2., fmax=40., average=True, spatial_colors=False)

#### Using Morlet Wavelets


In [ ]:
frequencies = np.arange(7, 30, 3)
power = mne.time_frequency.tfr_morlet(epochs, n_cycles=2, return_itc=False,
                                      freqs=frequencies, decim=3)

In [ ]:
power.plot(['FEF'])

In [ ]:
# For analysis: https://mne.tools/dev/auto_tutorials/time-freq/plot_sensors_time_frequency.html#inter-trial-coherence


# define frequencies of interest (log-spaced)
freqs = np.logspace(*np.log10([6, 35]), num=8)
n_cycles = freqs / 5.  # different number of cycle per frequency

power, itc = tfr_morlet(epochs, freqs=freqs, n_cycles=n_cycles, use_fft=True,
                        return_itc=True, decim=3, n_jobs=1, output= 'power')

#### Plot Power from Wavelet Analysis

In [ ]:
#power.plot_topo(baseline=(-0.5, 0), mode='logratio', title='Average power')
power.plot([1], baseline=(-0.5, 0), mode='logratio',)

In [ ]:
power.plot_joint(baseline=(-0.5, 0), mode='mean', tmin=-.5, tmax=2,
                 timefreqs=[(.5, 10), (1.3, 8)])

In [ ]:
raw2.info

In [ ]:
# inspect powerplot: https://mne.tools/0.15/auto_tutorials/plot_sensors_time_frequency.html

In [ ]:
power.plot_topo(baseline=(-0.5, 0), mode='logratio', title='Average power')
power.plot([82], baseline=(-0.5, 0), mode='logratio', title=power.ch_names[82])

fig, axis = plt.subplots(1, 2, figsize=(7, 4))
power.plot_topomap(ch_type='grad', tmin=0.5, tmax=1.5, fmin=8, fmax=12,
                   baseline=(-0.5, 0), mode='logratio', axes=axis[0],
                   title='Alpha', vmax=0.45, show=False)
power.plot_topomap(ch_type='grad', tmin=0.5, tmax=1.5, fmin=13, fmax=25,
                   baseline=(-0.5, 0), mode='logratio', axes=axis[1],
                   title='Beta', vmax=0.45, show=False)
mne.viz.tight_layout()
plt.show()